### Environment setup
```!pip install openai numpy tqdm tiktoken```

### Import Packages

In [1]:
from openai import OpenAI
import numpy as np
import pandas as pd
import concurrent.futures
from typing import List, Tuple, Dict
import os
import json
import random
from tqdm import tqdm
import tiktoken
import re
from typing import List, Dict, Tuple, Any
from heapq import nlargest
import csv

### Utility Functions

In [3]:
def clean_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r"\$", "", text)
    text = re.sub(r"(?s).*#### ", "", text)
    text = re.sub(r"\.$", "", text)
    text = re.sub(r",", "", text)
    
    if not text:
        return "-1000000000"
    if "." in text:  
        try:
            return str(int(float(text)))  # Convert "4.0" to "4"
        except ValueError:
            pass
    if text.isdigit():  
        return text

    return text

def extract_value(text: str) -> str:
    pattern = r"(-?[$0-9.,]{2,})|(-?[0-9]+)"
    matches = re.findall(pattern, text)
    
    if matches:
        for match_groups in matches[::-1]:
            for group in match_groups:
                if group:
                    return clean_text(group)
    
    return "-1000000000"

def load_dataset(file_path: str, sample_size: int = 20) -> List[Dict]:
    """Load and sample from dataset."""
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return random.sample(data, sample_size)

def count_tokens(text: str, model: str) -> int:
    """Count tokens in text using tiktoken."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

def evaluate_accuracy(predictions: List[str], ground_truth: List[str]) -> float:
    """Calculate accuracy of predictions."""
    correct = 0
    for pred, truth in zip(predictions, ground_truth):
        try:
            if int(pred) == int(truth):
                correct += 1
        except:
            pass
    return correct / len(predictions)

### Chain-of-Thoughts Implementation

1. Visit [DeepInfra](https://deepinfra.com/) and **register an account**. Familiarize yourself with how to use the API by referring to the [documentation](https://deepinfra.com/docs).  

2. Test the **API call** functionality provided by DeepInfra to ensure proper integration.


In [7]:
class ChainOfThought:
    def __init__(self, api_key: str, base_url: str = "https://api.deepinfra.com/v1/openai",
                 model: str = "Qwen/Qwen2.5-7B-Instruct", temperature: float = 0.3):
        self.client = OpenAI(
            api_key=api_key,
            base_url=base_url
        )
        self.model = model
        self.temperature = temperature
        self.total_tokens = 0

    def solve(self, question: str) -> Tuple[str, int]:
        prompt = f"""You are an excellent math problem solver. Your task is to solve the following math problem step by step. 
        Follow these instructions carefully:

        1. Understand the Problem: Read the problem carefully and identify what is being asked. Break down the problem into smaller parts if necessary.
        2. Plan the Solution: Decide on the mathematical concepts, formulas, or strategies that will be used to solve the problem.
        3. Execute the Plan: Perform the necessary calculations step by step. Introduce intermediate variables or assumptions if needed.
        4. Verify Each Step: After each step, explain the reasoning process of each step, check the correctness and confidence of the calculations and reasoning. If it's wrong, correct any errors immediately.
        5. Reflect on the Solution: After solving the problem, review the steps to ensure the solution is logical and accurate. Summarize the reasoning process.

        Problem: {question}

        Let's think step by step and ensure each step is correct before moving to the next one.
        """
        try:
            messages = [{"role": "user", "content": prompt}]
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=self.temperature,
                max_completion_tokens=2048,
            )
            tokens_used = response.usage.total_tokens
            self.total_tokens += tokens_used
            return response.choices[0].message.content, tokens_used
        except Exception as e:
            print(f"Error in API call: {e}")
            return "", 0

### Tree-of-Thoughts Implementation

1. You are *highly encouraged* to read the [original paper](http://arxiv.org/abs/2501.02497) and run the [codebase](https://github.com/princeton-nlp/tree-of-thought-llm) first.  
    - Otherwise, you may have no idea what ToT is doing.
    
    - For simplicity, start by running the basic configuration:
        - Search algorithm: BFS

        - Thought generator: propose prompt

        - Task: Game of 24

2. Regarding your own implementation below, feel free to experiment with various hyperparameters, including:
    - API call parameters (e.g., `temperature`)
    
    - ToT implementation parameters (e.g., `max_steps`, `n_samples_per_step`)

3. [Optional] You could try other **7B-level** base models instead of `Qwen2.5-7B-Instruct` in DeepInfra. You might achieve better results with proper implementation.

4. [Optional] Multi-model setups are also allowed—for example, using one model to generate thoughts and another to evaluate them (reward model?).

In [9]:
from typing import List, Dict, Tuple
import openai
from heapq import nlargest
from openai import OpenAI

class TreeOfThoughts:
    def __init__(self, api_key: str, base_url: str = "https://api.deepinfra.com/v1/openai",
                 model: str = "Qwen/Qwen2.5-7B-Instruct", temperature: float = 0.3):
        """Initialize the Tree-of-Thoughts solver."""
        # TODO: Initialize the OpenAI client and other necessary attributes
        self.client = OpenAI(
            api_key=api_key,
            base_url=base_url
        )
        self.model = model
        self.temperature = temperature
        self.evaluation_cache: Dict[str, float] = {}
        self.total_tokens = 0

    def chat_with_gpt(self, prompt: str, n: int = 1, stop: str = None) -> Tuple[List[str], int]:
        """Get completions from GPT model."""
        # [IMPORTANT] `stop` is important here. You can refer to the implementation of the Game of 24 in the original ToT codebase.
        # TODO: Implement the chat completion function
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[{"role": "user", "content": prompt}],
                temperature=self.temperature,
                n=n,
                max_completion_tokens=2048,
                stop=stop
            )
            tokens_used = response.usage.total_tokens
            self.total_tokens += tokens_used
            return [choice.message.content for choice in response.choices], tokens_used
        except Exception as e:
            print(f"Error in chat_with_gpt: {e}")
            return [], 0

    def generate_thoughts(self, question: str, current_thought: str = "", n_samples: int = 3) -> List[str]:
        """Generate multiple possible next steps in reasoning."""
        # [IMPORTANT] A one-shot example can help the model follow your instructions precisely.
        # TODO: Implement thought generation function
        prompt = f"""
        You are an excellent math problem solver. Solve the following problem with precise step-by-step reasoning.
        
        Problem: {question}
        Current Thought Process: {current_thought}

        Follow these instructions carefully:

        1. Understand the Problem: Read the problem carefully and identify what is being asked. Break down the problem into smaller step-by-step calculations if necessary. Clearly define the given information.
        2. Plan the Solution: Identify the required solution and relevant formulas. Decide on the mathematical concepts, formulas, or strategies that will be used to solve the problem.
        3. Execute the Plan: Perform the necessary calculations step by step accurately. Introduce intermediate variables or assumptions if needed. Output the final numerical answer in the correct format: just the number without additional text.
        4. Verify Each Step: After each step, explain the reasoning process of each step, check the correctness and confidence of the calculations and reasoning. If it's wrong, correct any errors immediately.
        5. Reflect on the Solution: After solving the problem, review the steps to ensure the solution is logical and accurate. Summarize the reasoning process.

        
        Let's think step by step strictly and ensure each step is correct before moving to the next one.
        """
        thoughts, tokens_used = self.chat_with_gpt(prompt, n=n_samples)
        self.total_tokens += tokens_used
        return thoughts

    def evaluate_thought(self, question: str, thought: str) -> float:
        """Evaluate the likelihood that a thought process leads to the correct answer."""
        # [IMPORTANT] You can use the base model (Qwen2.5-7B-Instruct) for self-evaluation; however, it is not the only option.
        # TODO: Implement thought evaluation function
        if thought in self.evaluation_cache:
            return self.evaluation_cache[thought]
        
        prompt = f"""Rate how likely this reasoning process will lead to the correct solution:
        Question: {question}
        Reasoning: {thought}
        Rate from 0 to 1, where 0 = wrong approach, 1 = correct approach.
        Return only the number."""
        
        response, _ = self.chat_with_gpt(prompt, n=1)
        try:
            score = float(response[0].strip())
            score = max(0.0, min(1.0, score))  # Clamp score between 0 and 1
            self.evaluation_cache[thought] = score
            return score
        except:
            return 0.0

    def select_best_thoughts(self, thoughts: List[str], scores: List[float], k: int = 2) -> List[str]:
        """Select the k best thoughts based on their scores."""
        # TODO: Implement thought selection function
        return [thought for thought, _ in nlargest(k, zip(thoughts, scores), key=lambda x: x[1])]

    def self_correct(self, question: str, solution: str) -> str:
        """Prompt the model to self-correct its solution."""
        prompt = f"""Review the following solution, reasoning, summary and correct any mistakes:
        Question: {question}
        Solution: {solution}
        
        Instructions:
        - Identify and fix any calculation errors.
        - Ensure all steps follow a logical sequence.
        - Ensure the final numerical answer is properly rounded.
        - Output only the corrected final numerical answer without additional text.
        """
        corrected_solution, _ = self.chat_with_gpt(prompt, n=1)
        return corrected_solution[0] if corrected_solution else solution

    
    
    def solve(self, question: str, max_steps: int = 8, n_samples_per_step: int = 3, k_best_thoughts: int = 2) -> str:
        """Solve a problem using Tree-of-Thoughts reasoning."""
        # TODO: Implement the main solving function
        # current_thoughts = ["Initial thought: Let's solve this step by step."]
        current_thoughts=['''You are an excellent math problem solver. Solve the following problem with precise step-by-step reasoning. Let's think step by step strictly and ensure each step is correct before moving to the next one.''']
        for step in range(max_steps):
            new_thoughts = []
            for thought in current_thoughts:
                generated_thoughts = self.generate_thoughts(question, thought, n_samples_per_step)
                new_thoughts.extend([
                    f"{thought}\nStep {step + 1}: {new_thought}"
                    for new_thought in generated_thoughts
                ])
            
            if not new_thoughts:
                break
            
            scores = [self.evaluate_thought(question, thought) for thought in new_thoughts]
            current_thoughts = self.select_best_thoughts(new_thoughts, scores, k_best_thoughts)
            
            if step == max_steps - 1:  # Only correct at the final step
                current_thoughts = [self.self_correct(question, thought) for thought in current_thoughts]
            
            if max(scores, default=0) > 0.95:
                return current_thoughts[0]
        
        return current_thoughts[0]

### Test ToT Using One Simple Example

In [10]:
def test_tot():
    # Set your API key
    api_key = os.getenv("DEEPINFRA_TOKEN", "")
    if not api_key:
        print("Please set DEEPINFRA_TOKEN environment variable")
        return

    # Test with a single example
    test_question = "If John has 5 apples and buys 3 more, how many apples does he have?"
    
    tot_solver = TreeOfThoughts(api_key)
    solution = tot_solver.solve(
        question=test_question,
        max_steps=20,
        n_samples_per_step=4,
        k_best_thoughts=2
    )
    
    answer = extract_value(solution)
    print("Solution:", solution)
    print("Extracted answer:", answer)

test_tot()

Solution: You are an excellent math problem solver. Solve the following problem with precise step-by-step reasoning. Let's think step by step strictly and ensure each step is correct before moving to the next one.
Step 1: ### Step-by-Step Solution

1. **Understand the Problem:**
   - John initially has 5 apples.
   - John buys 3 more apples.
   - We need to find out how many apples John has in total.

2. **Plan the Solution:**
   - We will use simple addition to find the total number of apples.
   - The formula is: Total apples = Initial apples + Apples bought.

3. **Execute the Plan:**
   - Initial apples = 5
   - Apples bought = 3
   - Total apples = 5 + 3

4. **Perform the Calculation:**
   - 5 + 3 = 8

5. **Verify Each Step:**
   - Initial apples (5) is correct.
   - Apples bought (3) is correct.
   - Adding 5 and 3 gives 8, which is correct.

6. **Reflect on the Solution:**
   - The steps are logical and the calculation is straightforward.
   - The final answer is 8.

**Final Answ

### Experiment on the Validation Set

- You can use multithreading to accelerate the process.

In [12]:
# 1. load DeepInfra api key
api_key = os.getenv("DEEPINFRA_TOKEN", "")

# 2. Load and sample dataset
dataset = load_dataset("cs5260_val_random300.jsonl", sample_size=100)

# 3. Initialize reasoning solvers
tot_solver = TreeOfThoughts(api_key)
cot_solver = ChainOfThought(api_key)


def process_batch(batch_data):
    tot_results, cot_results = [], []
    tot_tokens, cot_tokens = 0, 0
    
    for item in batch_data:
        question = item["question"]
        true_answer = item["answer"]
        
        # ToT solving
        tot_solution = tot_solver.solve(
            question=question,
            max_steps=8, 
            n_samples_per_step=4, 
            k_best_thoughts=2 
        )
        tot_answer = extract_value(tot_solution)
        tot_results.append({
            "question_id": item["question_id"],
            "predicted": tot_answer,
            "true": true_answer,
        })
            
        # # Check if ToT answer is incorrect and print details
        # if tot_answer != true_answer:
        #     print(f"ToT Answer Mismatch - Question ID: {item['question_id']}")
        #     print(f"Question: {question}")
        #     print(f"Correct Answer: {true_answer}")
        #     print(f"Generated Answer: {tot_answer}")
        #     print(f"Generated Answer Token Count: {len(tot_solution)}")  # Assuming 'tot_solution' is a list or string containing tokens
        
        # CoT solving
        cot_solution, tokens = cot_solver.solve(question)
        cot_answer = extract_value(cot_solution)
        cot_results.append({
            "question_id": item["question_id"],
            "predicted": cot_answer,
            "true": true_answer,
        })
        cot_tokens += tokens
            
        # # Check if CoT answer is incorrect and print details
        # if cot_answer != true_answer:
        #     print(f"CoT Answer Mismatch - Question ID: {item['question_id']}")
        #     print(f"Question: {question}")
        #     print(f"Correct Answer: {true_answer}")
        #     print(f"Generated Answer: {cot_answer}")
        #     print(f"Generated Answer Token Count: {tokens}")  # Assuming 'tokens' is the number of tokens generated
    
    return tot_results, cot_results, tot_tokens, cot_tokens


# 5. Process dataset using multithreading
batch_size = 10
num_threads = 10
batch_data = [dataset[i:i + batch_size] for i in range(0, len(dataset), batch_size)]

tot_results, cot_results = [], []
tot_tokens, cot_tokens = 0, 0

# Use ThreadPoolExecutor to process the batches in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(process_batch, batch) for batch in batch_data]
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
        batch_tot_results, batch_cot_results,  batch_tot_tokens, batch_cot_tokens = future.result()
        
        tot_results.extend(batch_tot_results)
        cot_results.extend(batch_cot_results)
        tot_tokens += batch_tot_tokens
        cot_tokens += batch_cot_tokens

# Calculate metrics
tot_accuracy = evaluate_accuracy([r["predicted"] for r in tot_results], 
                                [r["true"] for r in tot_results])
cot_accuracy = evaluate_accuracy([r["predicted"] for r in cot_results], 
                                [r["true"] for r in cot_results])

# Print results
print("\n=== Validation Results ===")
print(f"ToT Accuracy: {tot_accuracy:.2%}")
print(f"CoT Accuracy: {cot_accuracy:.2%}")
print(f"ToT Total Tokens: {tot_solver.total_tokens}")
print(f"CoT Total Tokens: {cot_tokens}")

100%|███████████████████████████████████████████| 10/10 [03:48<00:00, 22.80s/it]


=== Validation Results ===
ToT Accuracy: 91.00%
CoT Accuracy: 85.00%
ToT Total Tokens: 733225
CoT Total Tokens: 81181


### Submission

- Refer to [here](https://www.kaggle.com/competitions/cs-5260-spring-2025-assignment-1/data) for submission format

In [74]:
# TODO: Evaluate on test set and build submission file.

In [14]:
import pandas as pd
import concurrent.futures
from typing import List, Dict, Any
import csv

def evaluate_test_set(
    test_filepath: str,
    tot_solver: TreeOfThoughts,
    output_filepath: str = "sample_submission.csv",
    num_threads: int = 30
) -> None:
    print("\nProcessing test set...")
    test_data = load_dataset(test_filepath, sample_size=300)
    
    # Verify test data format
    if not all(item["question_id"].startswith("test_") for item in test_data):
        raise ValueError("Invalid test data format: question_ids should start with 'test_'")
    
    # Create batches of data
    batch_size = len(test_data) // num_threads
    batches = [test_data[i:i + batch_size] for i in range(0, len(test_data), batch_size)]
    
    results = []
    
    # Function to process a batch of questions
    def process_batch(batch_data):
        batch_results = []
        for item in batch_data:
            question = item["question"]
            question_id = item["question_id"]
            
            try:
                # Get ToT solution
                tot_solution = tot_solver.solve(
                    question=question,
                    max_steps=8,
                    n_samples_per_step=3,
                    k_best_thoughts=2
                )
                tot_answer = extract_value(tot_solution)
                
                # Handle None/invalid answers
                if tot_answer is None:
                    print(f"Warning: Could not extract numerical answer for {question_id}")
                    tot_answer = 0  # Default value for invalid answers
                
                batch_results.append({
                    "question_id": question_id,
                    "answer": tot_answer
                })
                
            except Exception as e:
                print(f"Error processing {question_id}: {str(e)}")
                # Include failed questions with default value
                batch_results.append({
                    "question_id": question_id,
                    "answer": 0
                })
        
        return batch_results

    # Use ThreadPoolExecutor to process batches in parallel
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [executor.submit(process_batch, batch) for batch in batches]
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            results.extend(future.result())
    
    # Convert to DataFrame and save as CSV
    df = pd.DataFrame(results)
    df.to_csv(output_filepath, index=False)
    
    print(f"\nTest set evaluation complete. Results saved to {output_filepath}")
    print(f"Total tokens used: {tot_solver.total_tokens}")
    print(f"Total questions processed: {len(results)}")
    
    # Verify submission format
    verify_submission_format(output_filepath)

def verify_submission_format(filepath: str) -> None:
    """
    Verify that the submission file meets the required format.

    Args:
        filepath: Path to the submission CSV file
    """
    df = pd.read_csv(filepath)
    
    # Check column names
    required_columns = ["question_id", "answer"]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Submission must contain columns: {required_columns}")
    
    # Check question_id format
    if not all(str(qid).startswith("test_") for qid in df["question_id"]):
        raise ValueError("All question_ids must start with 'test_'")
    
    # Check for duplicate question_ids
    if len(df["question_id"]) != len(df["question_id"].unique()):
        raise ValueError("Duplicate question_ids found in submission")
    
    # Check that answers are numeric
    if not pd.to_numeric(df["answer"], errors="coerce").notnull().all():
        raise ValueError("All answers must be numeric values")
    
    print("Submission format verification passed!")

# Evaluate on test set and create submission file
test_filepath = "cs5260_test_random300.jsonl"
output_filepath = "sample_submission.csv"
evaluate_test_set(test_filepath, tot_solver, output_filepath)


Processing test set...


100%|███████████████████████████████████████████| 30/30 [05:32<00:00, 11.08s/it]


Test set evaluation complete. Results saved to sample_submission.csv
Total tokens used: 2736064
Total questions processed: 300
Submission format verification passed!
